In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
ratings = pd.read_csv("/content/drive/MyDrive/ratings.csv")
print(ratings.head())

   userId  movieId  rating   timestamp
0     877     4155       5  1651201566
1     305     7661       2  1639553712
2     381     8423       2  1610704432
3     208     6433       1  1650223767
4      47     7752       4  1663998365


In [3]:
movies = pd.read_csv("/content/drive/MyDrive/movies.csv")
print(movies.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


#Data Analysis
calculate
total number of ratings
unique movies
unique users
the average number of ratings per user and per movie

In [4]:
n_ratings = len(ratings)
n_movies = len(ratings['movieId'].unique())
n_users = len(ratings['userId'].unique())

print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")
print(f"Number of unique users: {n_users}")
print(f"Average ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average ratings per movie: {round(n_ratings/n_movies, 2)}")

Number of ratings: 100836
Number of unique movieId's: 9742
Number of unique users: 999
Average ratings per user: 100.94
Average ratings per movie: 10.35


calculate the number of ratings each user

In [5]:
user_freq = ratings[['userId', 'movieId']].groupby(
    'userId').count().reset_index()
user_freq.columns = ['userId', 'n_ratings']
print(user_freq.head())

   userId  n_ratings
0       1        120
1       2        105
2       3         89
3       4        100
4       5        107


average ratings of each movie
identifies the highest and lowest rated movies

In [9]:
mean_rating = ratings.groupby('movieId')[['rating']].mean()
lowest_rated = mean_rating['rating'].idxmin()
movies.loc[movies['movieId'] == lowest_rated]
highest_rated = mean_rating['rating'].idxmax()
movies.loc[movies['movieId'] == highest_rated]
ratings[ratings['movieId']==highest_rated]
ratings[ratings['movieId']==lowest_rated]

movie_stats = ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()
movie_stats = movie_stats.sort_values(by='mean', ascending=False)
movie_stats

,count,mean
movieId,,
7831,5,4.800000
8682,4,4.750000
8966,5,4.600000
5002,7,4.571429
9270,2,4.500000
...,...,...
2981,4,1.250000
9399,4,1.250000
5519,4,1.000000


Highest rated movie

In [11]:
movies.iloc[7831]

,7831
movieId,93134
title,"Women on the 6th Floor, The (Les Femmes du 6èm..."
genres,Comedy


#Build Reco System

In [12]:
from scipy.sparse import csr_matrix

def create_matrix(df):

    N = len(df['userId'].unique())
    M = len(df['movieId'].unique())

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))

    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))

    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]

    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))

    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)

In [13]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    neighbour_ids = []

    if movie_id not in movie_mapper:
        print(f"Movie ID {movie_id} not found in movie_mapper!")
        return []

    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k += 1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1, -1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)

    for i in range(0, k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])

    neighbour_ids.pop(0)
    return neighbour_ids

In [22]:
def recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10):
    # Get ratings by the user
    df1 = ratings[ratings['userId'] == user_id]

    if df1.empty:
        print("No ratings found for this user.")
        return

    # Pick one of the highest-rated movies by the user
    top_rated = df1[df1['rating'] == df1['rating'].max()]
    movie_id = top_rated['movieId'].iloc[0]

    # Map movieId to title
    movie_titles = dict(zip(movies['movieId'], movies['title']))
    movie_title = movie_titles.get(movie_id, "Unknown Title")

    # Get similar movie internal IDs
    movie_index = movie_mapper[movie_id]
    similar_indices = find_similar_movies(movie_index, X, k)

    # Convert internal indices back to movie IDs
    similar_movie_ids = [movie_inv_mapper[i] for i in similar_indices]

    print(f"Since you liked **{movie_title}**, you might also like:")

    for i in similar_movie_ids:
        if i in movie_titles:
            print(f"- {movie_titles[i]}")


In [23]:
user_id = 150
recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10)

Since you liked **Miller's Crossing (1990)**, you might also like:
- Stunt Man, The (1980)
- Fried Green Tomatoes (1991)
- I Spy (2002)
- Soft Fruit (1999)
- Blind Fury (1989)
- Red Badge of Courage, The (1951)
- Laurel Canyon (2002)
